In [1]:
#!sudo /opt/conda/bin/conda install conda-build -y

In [2]:
#!sudo /opt/conda/bin/conda-develop -n QML-QPF PATH /workspaces/QML-QPF/mosaiQue

In [3]:

#import tensorflow as tf
#from tensorflow import keras
import mosaique as mq
from concurrent.futures import ProcessPoolExecutor, as_completed
import itertools
import numpy as np
import pennylane as qml
import os
import time
from mosaique.models.operation import OperationLayer
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


def operation():
    dev = qml.device("default.qubit.tf", wires=4)
    @qml.qnode(dev, interface='tf')
    def cnot(inputs):

        qml.AngleEmbedding(inputs[:,...], wires=range(4), rotation='Y')

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])

        # Measurement producing 4 classical output values
        return [qml.expval(qml.PauliZ(j)) for j in range(4)]
    return cnot

2024-11-21 03:59:32.048808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 03:59:32.774020: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def preset():
    from tensorflow import keras
    mnist_dataset = keras.datasets.mnist
    tr_layer = mq.ConvolutionLayer4x4("mnist_train")
    te_layer = mq.ConvolutionLayer4x4("mnist_test")
    (tr_images, tr_labels), (te_images, te_labels) = mnist_dataset.load_data()
    tr_layer.fit(tr_images)
    te_layer.fit(te_images)
    tr_images = tr_layer.transform(tr_images)
    te_images = te_layer.transform(te_images)
    return ((tr_layer, (tr_images, tr_labels)), (te_layer,(te_images, te_labels)))

with ProcessPoolExecutor() as executor:
    future = executor.submit(preset)

((train_layer, (train_images, train_labels)), (test_layer,(test_images, test_labels))) = future.result() #This blocks until the task completes

In [5]:
permutations = np.asarray(list(itertools.permutations(range(4))))

def pool(x, call, p, l):
    #sleep_duration = np.random.randint(1, 5) # Sleep between 1 and 4 seconds
    #time.sleep(sleep_duration)
    op = OperationLayer(call())
    predict = l.post_transform(op.pre_op.predict(x,batch_size=1000))
    l.save(predict, p)



In [6]:
#[pool(x=train_images[:,:,p], call=operation, p=p, l=train_layer) for p in permutations[:1]]


In [7]:
 
for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=train_images[:,:,p], call=operation, p=p, l=train_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#3 min 43 sec

2024-11-21 03:59:34.806519: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 03:59:34.806803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6795 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-21 03:59:35.102195: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 03:59:35.102472: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-21 03:59:35.405188: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 03:59:35.405479: I tensorflow/core/common_runtime/gpu/gpu

 2/60 ━━━━━━━━━━━━━━━━━━━━ 12s 217ms/step

I0000 00:00:1732161576.711153  945937 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-21 03:59:36.736984: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 03:59:36.737278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5045 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
I0000 00:00:1732161576.739710  946081 service.cc:145] XLA service 0x7707f0002350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732161576.739741  946081 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-11-21 03:59:36.777398: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_

 1/60 ━━━━━━━━━━━━━━━━━━━━ 1:48 2s/step

I0000 00:00:1732161577.079409  945983 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-21 03:59:37.198527: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 03:59:37.198866: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4625 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5


...

 1/60 ━━━━━━━━━━━━━━━━━━━━ 4:45 5s/step 1s/step 

I0000 00:00:1732161731.510107  950528 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 5/60 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/stept

I0000 00:00:1732161731.920033  950606 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step


In [8]:


for j in range(3):
    with ProcessPoolExecutor(8) as executor:
        runner = {
            executor.submit(pool, x=test_images[:,:,p], call=operation, p=p, l=test_layer): p for p in permutations[8*j:8*(j+1)]
        }
        for future in as_completed(runner):
            runner.pop(future)
#50 sec

2024-11-21 04:03:28.570756: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 04:03:28.571065: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6795 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-21 04:03:28.669644: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 04:03:28.670305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6643 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5
2024-11-21 04:03:28.742181: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-11-21 04:03:28.742703: I tensorflow/core/common_runtime/gpu/gpu

 1/10 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step

I0000 00:00:1732161811.554938  952314 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step

I0000 00:00:1732161812.158822  952358 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

 1/10 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/stepep

I0000 00:00:1732161847.647085  954410 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1/10 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/stepep

I0000 00:00:1732161848.075461  954505 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step
